# Parameter Management

In [1]:
%load_ext autoreload
%autoreload 2

from finestflow.base import Composable

In [2]:
class Plus(Composable):
    x: int
    y: int

    def run(self, z):
        return (self.x + self.y) * z

class Step2(Composable):
    a: int
    b: int

    def run(self):
        return self.a * self.b


def step3(a, b):
    return a + b


class Step4:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def calculate(self, c):
        return self.a + self.b + c

    
from typing import Union, Optional
class Workflow(Composable):
    param1: int
    param2: int
    step5: Composable
    step6: Union[Composable, int]
    step7: Optional[Composable]

    def _initialize_nodes(self):
        self.step1 = Plus(x=10, y=20)
        self.step2 = Step2(a=self.param1, b=self.param2)
        self.step3 = step3
        self.step4 = Step4(10, 10)

    def run(self, x):
        a = self.step1(x, _ff_name="step1")
        a += self.step2(_ff_name="step2")
        a += self.step3(a, x, _ff_name="step3")
        a += self.step4.calculate(x, _ff_name="step4")
        a += self.step5(a, x, _ff_name="step5")
        return a

class Workflow2(Workflow):
    param1: int
    param3: str
    _keywords = ["last_run", "hehe"]


In [3]:
params, nodes = Workflow2._collect_registered_params_and_nodes()

In [4]:
params

['param1', 'param2', 'param3']

In [5]:
nodes

['step5', 'step6', 'step7']

In [6]:
class Plus(Composable):
    y: int
    x: int = 10

    def run(self, z):
        return (self.x + self.y) * z
Plus.__dict__


Workflow.__dict__


workflow = Workflow(param1=2, param2=5, step5=step3)
repr(workflow)

'Workflow(param1=2, param2=5)'

In [11]:
workflow._info()

{'type': '__main__.Workflow',
 'params': {'param1': 2, 'param2': 5},
 'nodes': {'step5': {'type': 'finestflow.base.ComposableProxy',
   'params': {'ff_original_obj': '__main__.step3'},
   'nodes': {}},
  'step6': None,
  'step7': {'type': 'finestflow.base.ComposableProxy',
   'params': {'ff_original_obj': '__main__.step3'},
   'nodes': {}}}}

In [8]:
workflow.step8 = step3

In [10]:
workflow.step7 = step3

In [10]:
type(workflow.step7)

finestflow.base.ComposableProxy